## Etapa 2: Importação das bibliotecas

In [2]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

ImportError: DLL load failed while importing _imaging: Não foi possível encontrar o módulo especificado.

## Etapa 3: Construção da IA para negociação de ações

In [ ]:
class AI_Trader():
  
  def __init__(self, state_size, action_space = 3, model_name = "AITrader"):
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen = 2000)
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    self.model = self.model_builder()
    
  def model_builder(self):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, activation = "relu", input_dim = self.state_size))
    model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 128, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = self.action_space, activation = "linear"))
    model.compile(loss = "mse", optimizer = tf.keras.optimizers.Adam(lr = 0.001))
    return model
  
  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  def batch_train(self, batch_size):
    print ("Batch Train")
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay
      

## Etapa 4: Pré-processamento da base de dados

### Definição de funções auxiliares

#### Sigmoid

In [ ]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#### Formatação de preços

In [ ]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Carregador da base de dados

In [ ]:
def dataset_loader(stock_name):
  dataset = data_reader.DataReader(stock_name, data_source = "yahoo")
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close

### Criador de estados

In [ ]:
def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep + 1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))
    
  return np.array([state]), windowed_data

### Carregando a base de dados

In [ ]:
stock_name = "MSFT"
data = dataset_loader(stock_name)

## Etapa 5: Treinando a IA

### Configuração dos hyper parâmetros

In [ ]:
window_size = 10
episodes = 5
batch_size = 32
data_samples = len(data) - 1

In [ ]:
data_samples

1259

### Definição do modelo

In [ ]:
trader = AI_Trader(window_size)

In [ ]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Loop de treinamento

In [ ]:
for episode in range(1, episodes + 1):
  print("Episode: {}/{}".format(episode, episodes))
  state = state_creator(data, 0, window_size + 1)
  total_profit = 0
  trader.inventory = []
  for t in tqdm(range(data_samples)):
    action = trader.trade(state)
    next_state = state_creator(data, t + 1, window_size + 1)
    reward = 0
    
    if action == 1: # Comprando uma ação
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
    elif action == 2 and len(trader.inventory) > 0: # Vendendo uma ação  
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price))
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("Total profit: {}".format(total_profit))
      print("########################")
    
#    if len(trader.memory) > batch_size:
#      trader.batch_train(1)
    
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  1%|          | 15/1259 [00:00<00:08, 138.86it/s]

Episode: 1/5
AI Trader bought:  $ 54.709999
AI Trader sold:  $ 55.049999  Profit: $ 0.340000
AI Trader bought:  $ 55.570000
AI Trader sold:  $ 54.560001  Profit: - $ 1.009998
AI Trader bought:  $ 54.310001
AI Trader bought:  $ 54.650002
AI Trader bought:  $ 55.349998
AI Trader sold:  $ 55.360001  Profit: $ 1.049999
AI Trader bought:  $ 55.650002
AI Trader sold:  $ 56.389999  Profit: $ 1.739998
AI Trader bought:  $ 55.779999
AI Trader sold:  $ 52.110001  Profit: - $ 3.239998
AI Trader sold:  $ 51.439999  Profit: - $ 4.210003
AI Trader sold:  $ 50.939999  Profit: - $ 4.840000
AI Trader bought:  $ 49.869999
AI Trader bought:  $ 49.939999
AI Trader sold:  $ 50.389999  Profit: $ 0.520000
AI Trader bought:  $ 51.049999
AI Trader bought:  $ 51.509998
AI Trader bought:  $ 51.080002
AI Trader sold:  $ 50.509998  Profit: $ 0.570000
AI Trader sold:  $ 50.810001  Profit: - $ 0.239998
AI Trader bought:  $ 50.619999
AI Trader sold:  $ 51.590000  Profit: $ 0.080002
AI Trader bought:  $ 52.119999
AI T

 25%|██▌       | 317/1259 [00:00<00:03, 258.68it/s]

  Profit: - $ 0.789997
AI Trader sold:  $ 57.639999  Profit: $ 0.180000
AI Trader sold:  $ 57.799999  Profit: $ 0.189999
AI Trader sold:  $ 56.919998  Profit: $ 0.709999
AI Trader sold:  $ 57.220001  Profit: $ 0.170002
AI Trader bought:  $ 57.660000
AI Trader bought:  $ 57.250000
AI Trader bought:  $ 59.660000
AI Trader sold:  $ 61.000000  Profit: $ 4.470001
AI Trader sold:  $ 60.630001  Profit: $ 3.380001
AI Trader bought:  $ 60.099998
AI Trader bought:  $ 59.869999
AI Trader sold:  $ 59.799999  Profit: $ 2.989998
AI Trader bought:  $ 59.209999
AI Trader bought:  $ 60.419998
AI Trader bought:  $ 60.169998
AI Trader bought:  $ 58.700001
AI Trader bought:  $ 59.020000
AI Trader bought:  $ 58.869999
AI Trader bought:  $ 60.349998
AI Trader sold:  $ 60.400002  Profit: $ 2.970001
AI Trader bought:  $ 59.250000
AI Trader bought:  $ 60.220001
AI Trader bought:  $ 59.950001
AI Trader bought:  $ 61.369999
AI Trader sold:  $ 61.009998  Profit: $ 4.109997
AI Trader bought:  $ 61.970001
AI Trader

 51%|█████     | 641/1259 [00:00<00:01, 449.26it/s]

...

  Profit: $ 1.760002
AI Trader bought:  $ 93.519997
AI Trader sold:  $ 93.510002  Profit: - $ 0.259995
AI Trader sold:  $ 94.070000  Profit: $ 1.220001
AI Trader bought:  $ 95.160004
AI Trader sold:  $ 96.220001  Profit: $ 2.040001
AI Trader bought:  $ 96.940002
AI Trader bought:  $ 97.910004
AI Trader sold:  $ 97.699997  Profit: $ 4.809998
AI Trader bought:  $ 97.320000
AI Trader bought:  $ 97.150002
AI Trader bought:  $ 96.180000
AI Trader bought:  $ 97.599998
AI Trader bought:  $ 97.500000
AI Trader sold:  $ 98.660004  Profit: $ 6.180000
AI Trader sold:  $ 98.010002  Profit: $ 8.220001
AI Trader sold:  $ 98.839996  Profit: $ 9.369995
AI Trader bought:  $ 100.790001
AI Trader bought:  $ 102.190002
AI Trader sold:  $ 102.489998  Profit: $ 11.220001
AI Trader sold:  $ 100.879997  Profit: $ 8.549995
AI Trader sold:  $ 101.050003  Profit: $ 10.280006
AI Trader bought:  $ 100.849998
AI Trader bought:  $ 101.419998
AI Trader sold:  $ 100.860001  Profit: $ 7.980003
AI Trader bought:  $ 101.

 78%|███████▊  | 983/1259 [00:00<00:00, 1298.38it/s]


AI Trader sold:  $ 119.889999  Profit: $ 17.089996
AI Trader bought:  $ 120.190002
AI Trader sold:  $ 120.330002  Profit: $ 16.730003
AI Trader bought:  $ 120.949997
AI Trader sold:  $ 120.769997  Profit: $ 17.969994
AI Trader bought:  $ 121.769997
AI Trader sold:  $ 123.370003  Profit: $ 17.990005
AI Trader bought:  $ 123.760002
AI Trader sold:  $ 125.010002  Profit: $ 18.889999
AI Trader bought:  $ 129.149994
AI Trader bought:  $ 127.879997
AI Trader bought:  $ 126.209999
AI Trader sold:  $ 128.899994  Profit: $ 22.699997
AI Trader bought:  $ 128.149994
AI Trader sold:  $ 125.519997  Profit: $ 18.349998
AI Trader bought:  $ 125.510002
AI Trader sold:  $ 124.730003  Profit: $ 18.350006
AI Trader sold:  $ 126.019997  Profit: $ 23.239998
AI Trader sold:  $ 128.929993  Profit: $ 23.189995
AI Trader sold:  $ 127.669998  Profit: $ 20.449997
AI Trader sold:  $ 126.180000  Profit: $ 20.910004
AI Trader sold:  $ 124.940002  Profit: $ 19.690002
AI Trader bought:  $ 125.730003
AI Trader bought

100%|██████████| 1259/1259 [00:00<00:00, 1377.34it/s]


AI Trader sold:  $ 183.509995  Profit: $ 39.449997
AI Trader bought:  $ 181.570007
AI Trader bought:  $ 181.809998
AI Trader bought:  $ 181.399994
AI Trader bought:  $ 183.250000
AI Trader sold:  $ 182.830002  Profit: $ 38.570007
AI Trader bought:  $ 184.910004
AI Trader bought:  $ 185.360001
AI Trader sold:  $ 187.199997  Profit: $ 39.139999
AI Trader sold:  $ 196.839996  Profit: $ 46.449997
AI Trader sold:  $ 186.270004  Profit: $ 36.650009
AI Trader bought:  $ 187.740005
AI Trader sold:  $ 193.570007  Profit: $ 42.340012
AI Trader bought:  $ 194.240005
AI Trader sold:  $ 196.320007  Profit: $ 46.770004
AI Trader bought:  $ 195.149994
AI Trader bought:  $ 200.570007
AI Trader sold:  $ 197.839996  Profit: $ 47.989990
AI Trader bought:  $ 196.330002
AI Trader sold:  $ 203.509995  Profit: $ 53.580002
AI Trader bought:  $ 204.699997
AI Trader sold:  $ 206.259995  Profit: $ 54.899994
AI Trader sold:  $ 210.699997  Profit: $ 56.330002
AI Trader sold:  $ 208.250000  Profit: $ 50.869995
AI 